
<a id='chap-tpdeeplearning3'></a>

# Travaux pratiques - Deep Learning avec Keras

L’objectif de cette séance de travaux pratiques est de découvrir la bibliothèque
[Keras](https://keras.io/) et ses fonctionalités pour la création et l’apprentissage
de réseaux de neurones profonds. En particulier, nous allons reprendre les deux modèles
vus précédemment (régression logistique et perceptron) pour montrer comment les
implémenter avec Keras. Nous verrons également un premier modèle simple de réseau
de neurones profonds convolutif.

## Importation de Keras/TensorFlow

Keras est une bibliothèque logicielle open-source écrite en Python pour l’apprentissage profond.
Il s’agit en réalité d’une surcouche à [TensorFlow](https://www.tensorflow.org/?hl=fr), qui implémente
les briques de base (opérations matricielles, couches, fonctions de transfert). TensorFlow gère
notamment la possibilité d’exécuter un réseau de neurones sur le processeur (*CPU*) ou sur accélérateur
graphique matériel (*GPU*).

L’import de Keras s’effectue depuis TensorFlow :

In [55]:
!pip install --upgrade pybind11

In [56]:
from tensorflow import keras
import numpy as np

En plus des fonctionnalités liées à l’apprentissage profond, `Keras` intègre des utilitaires
pour charger et manipuler un certain nombre de jeux de données populaires, comme MNIST :

In [57]:
# Chargement du jeu de données MNIST
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()
# Conversion en encodage one-hot
Y_train = keras.utils.to_categorical(y_train, 10)
Y_test = keras.utils.to_categorical(y_test, 10)
# Redimensionnement des images 28x28 en vecteurs d=784
X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
# Assurez-vous que les images sont de type float pour la normalisation
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)
# Normalisation entre 0 et 1
X_train /= 255
X_test /= 255

X_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

## Régression logistique avec `Keras`

`Keras` propose plusieurs façons différentes de définir un réseau de neurones. La façon
la plus courante pour les réseaux à propagation avant (*feedforward*), qui empilent les couches
de façon séquentielle, est d’utiliser la classe Sequential (voir [sa documentation](https://keras.io/guides/sequential_model/)):

In [58]:
from tensorflow.keras import Sequential
model = Sequential()

`model` représente ainsi un réseau de neurones vide (pour l’instant). Il est possible
d’ajouter des couches à l’aide de la méthode `add`. De nombreuses couches sont
prédéfinies dans Keras, comme les couches entièrement connectées (couches linéaires
dites `Dense`) ou les fonctions d’activation standard.

Par exemple, le code ci-dessous ajoute une projection linéaire (couche entièrement
connectée) de taille 10 au modèle, puis une activation de type *softmax*:

In [59]:
from tensorflow.keras.layers import Dense, Activation

model.add(Dense(10,  input_dim=784, name='fc1'))
model.add(Activation('softmax'))

Le paramètre `input_dim` est nécessaire pour spécifier la dimension de l’entrée de la couche entièrement connectée.
On peut ensuite visualiser l’architecture du réseau à l’aide de la méthode `summary()` :

In [60]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 fc1 (Dense)                 (None, 10)                7850      
                                                                 
 activation_2 (Activation)   (None, 10)                0         
                                                                 
Total params: 7850 (30.66 KB)
Trainable params: 7850 (30.66 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Question

Vérifier le nombre de paramètres du réseau à apprendre dans la méthode `summary()`.

In [61]:
nb_parameters = model.summary()
print(nb_parameters)

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 fc1 (Dense)                 (None, 10)                7850      
                                                                 
 activation_2 (Activation)   (None, 10)                0         
                                                                 
Total params: 7850 (30.66 KB)
Trainable params: 7850 (30.66 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


En plus de la définition de l’architecture, nous avons encore besoin de spécifier deux éléments à `Keras` avant d’entraîner
notre modèle : une fonction de coût (*loss*) et une méthode d’optimisation.
Ces paramètres sont spécifiés lors de la phase de *compilation* du modèle à l’aide de la méthode `.compile()`.
Nous allons utiliser l’entropie croisée (`categorical_crossentropy`) comme fonction de coût et la descente de gradient stochastique
(*stochastic gradient descent* ou `sgd`).

In [62]:
optimizer = keras.optimizers.SGD(learning_rate=0.5)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

## Question

À l’aide de la [documentation de la méthode .compile()](https://keras.io/api/models/model_training_apis/), déterminer à quoi correspond le paramètre `metrics=`.

La boucle d’apprentissage du modèle sur les données d’apprentissage est automatisée par la méthode `.fit()` :

Réponse : Le paramètre metrics correspond à la liste des métriques devant être évaluées par le modèle lors de l'entrainement et de la validation. 

In [63]:
model.fit(X_train, Y_train, batch_size=300, epochs=10, verbose=1)

Epoch 1/10


2025-03-12 11:59:21.480519: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 188160000 exceeds 10% of free system memory.


200/200 [==============================] - 1s 2ms/step - loss: 0.4800 - accuracy: 0.8684
Epoch 2/10
200/200 [==============================] - 0s 2ms/step - loss: 0.3361 - accuracy: 0.9055
Epoch 3/10
200/200 [==============================] - 0s 2ms/step - loss: 0.3138 - accuracy: 0.9117
Epoch 4/10
200/200 [==============================] - 0s 2ms/step - loss: 0.3016 - accuracy: 0.9159
Epoch 5/10
200/200 [==============================] - 0s 2ms/step - loss: 0.2939 - accuracy: 0.9179
Epoch 6/10
200/200 [==============================] - 0s 2ms/step - loss: 0.2883 - accuracy: 0.9194
Epoch 7/10
200/200 [==============================] - 0s 2ms/step - loss: 0.2841 - accuracy: 0.9206
Epoch 8/10
200/200 [==============================] - 0s 2ms/step - loss: 0.2803 - accuracy: 0.9221
Epoch 9/10
200/200 [==============================] - 0s 2ms/step - loss: 0.2776 - accuracy: 0.9215
Epoch 10/10
200/200 [==============================] - 0s 2ms/step - loss: 0.2752 - accuracy: 0.9232


- `batch_size` correspond au nombre d’exemples utilisé pour estimer
  le gradient de la fonction de coût.  
- `epochs` est le nombre d’époques (*i.e.* passages sur l’ensemble
  des exemples de la base d’apprentissage) lors de la descente de
  gradient.  

## Note

Pour les fonctions de coût de classification, Keras attend par convention que les étiquettes
(*labels*) données pour la supervision soient au format *one-hot encoding*.

L’évaluation des performances du modèle sur le jeu de test est également automatique, en fonction de la métrique choisie lors de la compilation, grâce à la méthode `.evaluate()` :

In [64]:
scores = model.evaluate(X_test, Y_test, verbose=0)
print(f"{model.metrics_names[0]}: {scores[0]*100:.2f}")
print(f"{model.metrics_names[1]}: {scores[1]*100:.2f}")

loss: 27.43
accuracy: 92.24


Le premier élément de la liste `scores` correspond à l’erreur de test (valeur de la fonction de coût sur
`(X_test, y_test)`) tandis que le deuxième élément correspond au taux de bonne classification (*accuracy*).

## Question

Comparer les performances obtenues par cette régression logistique implémentée avec Keras avec
celles obtenues lors du premier TP. Conclure.

On remarque que cette régression logistique implémentée avec Keras obtient les mêmes résultats en termes d'accuracy que le test effectué sur le premier TP avec le premier réseau de neurones

## Perceptron avec Keras

Nous pouvons maintenant enrichir ce modèle de régression logistique en insérant
une couche de neurones cachés complètement connectée ainsi qu’une
fonction d’activation non linéaire de type sigmoïde entre la couche
d’entrée et la couche de sortie. Ce nouveau modèle va ainsi correspond
au réseau de neurones à une couche cachée (perceptron) vu précédemment.

## Question

Créer un nouveau modèle `model = Sequential()` et le compléter en lui
ajoutant :

- une couche entièrement connectée avec 100 neurones cachés,  
- une activation sigmoide,  
- une couche entièrement connectée avec 10 neurones de sortie,  
- une activation softmax,  


de sorte à retrouver le *multi-layer perceptron* du TP précédent.

In [65]:
# Création du modèle
from keras.layers import Dense
model = Sequential()

# Première couche entièrement connectée (100 neurones cachés, activation sigmoïde)
model.add(Dense(100, activation='sigmoid', input_shape=(784,)))

# Deuxième couche entièrement connectée (10 neurones de sortie, activation softmax)
model.add(Dense(10, activation='softmax'))

## Question

Combien ce modèle a-t-il de paramètres ? Justifier le calcul et vérifier à l’aide de la méthode `.summary()`.

In [66]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 100)               78500     
                                                                 
 dense_9 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79510 (310.59 KB)
Trainable params: 79510 (310.59 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Réponse : ce modèle comporte maintenant 79510 paramètres

L’entraînement de ce nouveau modèle s’effectue de façon strictement identique à ce que nous avons vu précédemment.
En effet, Keras s’inspire de l’interface de scikit-learn : tous les modèles se manipulent de la même façon une fois
compilés. L’algorithme de rétropropagation du gradient et l’optimisation par descente de gradient sont implémentés
dans Keras et il n’est plus nécessaire de les écrire à la main.

## Question

Compiler le perceptron multi-couche puis l’entraîner sur la base d’apprentissage de MNIST. On utilisera
la même fonction de coût et le même optimiseur que pour la régression logistique.

In [67]:
optimizer = keras.optimizers.SGD(learning_rate=0.5)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

## Question

Évaluer les performances du perceptron multi-couche sur le jeu de test de MNIST et comparer
à celles obtenues lors du précédent TP. Conclure.

In [68]:

scores_mlp = model.evaluate(X_test, Y_test, verbose=0)
print("Loss: %.2f%%" % (scores_mlp[0]*100))
print("Accuracy: %.2f%%" % (scores_mlp[1]*100))


Loss: 283.87%
Accuracy: 9.80%


Réponse : l'accuracy est de 12.38%, ce qui correspond aux valeurs obtenues sur les premiers epochs du tp2

## Question

À l’aide de la [documentation de Keras](https://keras.io/api/layers/core_layers/dense/), déterminer
comment sont initialisés les paramètres du modèle pour les couches entièrement connectées.

Une fonctionnalité utile de Keras est la possibilité de sauvegarder des modèles sur le disque pour les partager
ou les réutiliser plus tard. Cette opération se fait simplement à l’aide de la méthode `.save()` :

In [69]:
model.save('MLP.h5')

/home/florian-andr/anaconda3/envs/jpn_florian/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Il est possible de charger un modèle sur le disque à l’aide de la fonction `load_model` :

In [70]:
from tensorflow.keras.models import load_model
model = load_model('MLP.h5')

## Réseaux de neurones convolutifs avec Keras

Pour cette dernière partie, nous allons passer du réseau entièrement connecté à un modèle convolutif.
En effet, les réseaux de neurones convolutifs profonds (*Convolutional Neural Networks* ou CNN) sont
particulièrement adaptés à la reconnaissance d’images.

Les réseaux convolutifs manipulent des images multi-dimensionnelles en entrée (des tenseurs).
Comme nous avions initialement transformé nos images en vecteurs, nous allons les redimensionner
de sorte à ce que chaque observation du jeu de données MNIST soit bien une image carrée de 28 pixels
de côté en niveaux de gris, c’est-à-dire de dimensions $ 28\times 28\times 1 $.

In [71]:
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)

Par rapport aux réseaux complètement connectés, les réseaux convolutifs
utilisent les briques élémentaires suivantes :

1. Des couches de convolution, qui transforment un tenseur d’entrée de
  taille $ n_x \times n_y \times p $ en un tenseur de sortie
  $ n_{x'} \times n_{y'} \times n_H $, où $ n_H $ est le nombre
  de filtres choisi.  


Keras implémente bien entendu par défaut les couches convolutives. Il s’agit de la classe `Conv2D`.
Par exemple, une couche de convolution pour traiter les images d’entrée de MNIST peut être créée de la manière suivante :

In [72]:
from tensorflow.keras.layers import Conv2D
#Conv2D(..., kernel_size=(..., ...), activation=, input_shape=(input_shape), padding=...) 
Conv2D(32, kernel_size=(5, 5), activation='relu', input_shape=input_shape, padding='same')

Dans l’instanciation de cette classe, on retrouve de nombreux arguments :

- 32 correspond au nombre de filtres souhaité pour cette couche convolutive.  
- (5, 5) sont les dimensions spatiales du noyau de convolution de chacun des filtres.  
- `padding='same'` ajoute des 0 à l’image d’entrée de sorte à conserver la même taille en sortie qu’en entrée ($ n_x = n_x' $).  

## Note

On remarque qu’il est possible de spécifier directement dans la couche de convolution
la non-linéarité qui sera appliquée en sortie des activations. Dans notre cas,
nous avons choisi d’appliquer une fonction d’activation sigmoide après la convolution.

1. Des couches d’agrégation spatiale (*pooling*), afin de permettre une
  invariance aux translations locales. Elles sont implémentées par la classe `MaxPooling2D`
  de Keras :  

In [73]:
from tensorflow.keras.layers import MaxPooling2D
MaxPooling2D(pool_size=(2,2))

- (2, 2) est la taille spatiale sur laquelle l’opération d’agrégation
  est effectuée.  


Ces deux briques élémentaires peuvent être ajoutées à n’importe quel modèle séquentiel en Keras à l’aide de la méthode `.add()`.

## Question

Implémenter un réseau de neurones convolutif reprenant l’architecture suivante, similaire à celle du modèle historique LeNet5 [[LBD+89]](#lecun1989backpropagation),
illustré ci-dessous :

<img src="LeNet5.png" style="width:600;">

- Une couche de convolutions avec 16 filtres de taille
  $ 3 \times 3 $, suivie d’une non-linéarité sigmoïde puis
  d’une couche de max pooling $ 2 \times 2 $.  
- Une couche de convolutions avec 32 filtres de taille
  $ 3 \times 3 $, suivie d’une non-linéarité sigmoïde puis
  d’une couche de max pooling $ 2 \times 2 $.  
- Une seconde couche de convolution avec 64 filtres de taille
  $ 3 \times 3 $, suivie d’une non linéarité sigmoïde puis
  d’une couche de max pooling $ 2 \times 2 $.  
- Comme dans le réseau LeNet, on considérera la sortie du second bloc
  convolutif comme un vecteur, ce que revient à « mettre à plat » les
  couches convolutives précédentes (`model.add(Flatten())`).  
- Une couche complètement connectée de taille 100, suivie d’une sigmoide.  
- Une couche complètement connectée de taille 10, suivie d’une activation softmax.  

In [74]:
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten

# Création du modèle
model = Sequential()

# Premier bloc convolutif: 16 filtres de taille 3x3, activation sigmoïde, suivi de max pooling 2x2
model.add(Conv2D(16, (3, 3), activation='sigmoid', input_shape=(28, 28, 1), padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Deuxième bloc convolutif: 32 filtres de taille 3x3, activation sigmoïde, suivi de max pooling 2x2
model.add(Conv2D(32, (3, 3), activation='sigmoid', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Troisième bloc convolutif: 64 filtres de taille 3x3, activation sigmoïde, suivi de max pooling 2x2
model.add(Conv2D(64, (3, 3), activation='sigmoid', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Mise à plat (flatten) de la sortie des couches convolutives pour passer à une couche entièrement connectée
model.add(Flatten())

# Couche entièrement connectée de taille 100, activation sigmoïde
model.add(Dense(100, activation='sigmoid'))

# Couche de sortie, complètement connectée, 10 neurones, activation softmax pour la classification multiclasse
model.add(Dense(10, activation='softmax'))

## Question

Compiler puis entraîner le modèle sur le jeu d’apprentissage de MNIST. On reprendra la même fonction de coût et
le même optimiseur que précédemment.

In [75]:
# Compilation du modèle
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entraînement du modèle
model.fit(X_train, Y_train, epochs=10, batch_size=32)

Epoch 1/10
1875/1875 [==============================] - 15s 8ms/step - loss: 1.0247 - accuracy: 0.6381
Epoch 2/10
1875/1875 [==============================] - 14s 8ms/step - loss: 0.1276 - accuracy: 0.9617
Epoch 3/10
1875/1875 [==============================] - 15s 8ms/step - loss: 0.0797 - accuracy: 0.9752
Epoch 4/10
1875/1875 [==============================] - 14s 8ms/step - loss: 0.0612 - accuracy: 0.9805
Epoch 5/10
1875/1875 [==============================] - 14s 8ms/step - loss: 0.0492 - accuracy: 0.9845
Epoch 6/10
1875/1875 [==============================] - 14s 8ms/step - loss: 0.0411 - accuracy: 0.9869
Epoch 7/10
1875/1875 [==============================] - 15s 8ms/step - loss: 0.0342 - accuracy: 0.9891
Epoch 8/10
1875/1875 [==============================] - 14s 8ms/step - loss: 0.0303 - accuracy: 0.9904
Epoch 9/10
1875/1875 [==============================] - 14s 8ms/step - loss: 0.0259 - accuracy: 0.9920
Epoch 10/10
1875/1875 [==============================] - 14s 8ms/step - l

In [76]:
model.save('LeNet.h5')

## Question

Évaluer les performances du modèle (classification_report) sur le jeu de test de MNIST. Vous devriez obtenir un score proche de 99% pour ce modèle.

In [77]:
from sklearn.metrics import classification_report
import numpy as np

# Faire des prédictions sur l'ensemble de test
y_pred = model.predict(X_test)

# Convertir les probabilités en classes prédictes (en prenant la classe avec la probabilité maximale --> np.argmax)
y_pred_classes = np.argmax(y_pred, axis=1)

# Convertir les étiquettes réelles en classes (en prenant l'index des classes)
y_true = np.argmax(Y_test, axis=1)

# Générer le rapport de classification avec précision, rappel et F1-score
report = classification_report(y_true, y_pred_classes)

print(report)

313/313 [==============================] - 1s 3ms/step
              precision    recall  f1-score   support

           0       1.00      0.99      0.99       980
           1       1.00      0.99      0.99      1135
           2       0.98      1.00      0.99      1032
           3       0.98      1.00      0.99      1010
           4       0.99      0.98      0.99       982
           5       0.99      0.98      0.99       892
           6       0.99      0.99      0.99       958
           7       0.99      0.99      0.99      1028
           8       0.98      0.98      0.98       974
           9       0.97      0.99      0.98      1009

    accuracy                           0.99     10000
   macro avg       0.99      0.99      0.99     10000
weighted avg       0.99      0.99      0.99     10000

